In [29]:
import kfp
from kfp import dsl
from kfp.dsl import component
from typing import Optional

@component(
    base_image='python:3.11.10',
    packages_to_install=['minio', 'pandas', 'openpyxl','numpy==1.24.4']
)
def get_X() -> int:
    import pandas as pd
    from minio import Minio

    # Initialize MinIO client
    client = Minio("10.111.218.179:9000", access_key="minio", secret_key="minio123", secure=False)

    # Download the CSV file
    response = client.fget_object("mlpipeline", "LEAF_LEVEL_DATASET_Yufeng_Ge.xlsx","file.xlsx")
   
    # Read and load dataset
    df= pd.read_excel("file.xlsx", sheet_name=[0, 1])

    X =df.get(0)
    X = (X.iloc[:,14:])
    X.to_csv("input_X.csv", index=False)

    client.fput_object("mlpipeline", "input_X.csv", "input_X.csv")
    return 0

@component(
    base_image='python:3.11.10',
    packages_to_install=['minio', 'pandas', 'openpyxl','numpy==1.24.4']
)
def get_Y() -> int:
    import pandas as pd
    from minio import Minio
    # Initialize MinIO client
    client = Minio("10.111.218.179:9000", access_key="minio", secret_key="minio123", secure=False)
   
    # Download the CSV file
    response = client.fget_object("mlpipeline", "LEAF_LEVEL_DATASET_Yufeng_Ge.xlsx","file.xlsx")
   

    # Read and load dataset
    df= pd.read_excel("file.xlsx", sheet_name=[0, 1])
    Y =df.get(0)
    Y = (Y.iloc[:,7])
    Y.to_csv("input_Y.csv", index=False)

    client.fput_object("mlpipeline", "input_Y.csv", "input_Y.csv")
    return 0



def NRMSEPiqr(observed_values, predicted_values):





@component(
    base_image='python:3.11.10',
    packages_to_install=['scikit-learn', 'pandas','minio','openpyxl','numpy==1.24.4']
)
def train_model(X: int, Y:int) -> str:
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import Ridge
    import pandas as pd
    from minio import Minio
    from joblib import dump
    client = Minio("10.111.218.179:9000", access_key="minio", secret_key="minio123", secure=False)
   
    # Download the CSV files
    response = client.fget_object("mlpipeline","input_X.csv", "input_X.csv")
    response = client.fget_object("mlpipeline","input_Y.csv", "input_Y.csv")

    X = df= pd.read_csv("input_X.csv")
    Y = df= pd.read_csv("input_Y.csv")
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=21)
    alpha = 0.26160739
    ridge_reg_pipeline = make_pipeline(StandardScaler(),Ridge(alpha=alpha))

    ridge_reg_pipeline.fit(X_train, Y_train)
    
    # Make predictions using the testing set
    Y_pred_ridge_cl =  ridge_reg_pipeline.predict(X_test)
    dump(ridge_reg_pipeline, 'model.joblib')
    client.fput_object("mlpipeline", 'models/leafRecognize/1/model.joblib', 'model.joblib')
    cv_scores = cross_validate(best_model, X_train, Y_train, cv=sturges, scoring=NRMSEPiqrscorer,n_jobs=-1)   

    # Calculate RMSEP
    rmsep = np.sqrt(np.mean((Y_test - Y_pred_ridge_cl) ** 2))
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = np.percentile(Y_test, 25)
    Q3 = np.percentile(Y_test, 75)

    # Calculate IQR
    IQR = Q3 - Q1

    return rmsep/IQR

    return str(NRMSEPiqr(Y_test, Y_pred_ridge_cl))

@component(
    base_image='python:3.11.10',
    packages_to_install=[
        'kserve',
        'numpy==1.24.4',
        'pandas'
    ]
)
def create_inference_service(
    model_uri: str,
    model_name: str,
    namespace: str = "kubeflow-user-example-com",
    min_replicas: Optional[int] = None,
    max_replicas: Optional[int] = None
) -> None:
    """
    Deploy a model using KServe.

    Args:
        model_uri (str): URI of the model to deploy.
        model_name (str): Name for the deployed model.
        namespace (str): Kubernetes namespace for deployment.
        min_replicas (int, optional): Minimum number of replicas.
        max_replicas (int, optional): Maximum number of replicas.

    Raises:
        Exception: If deployment fails.
    """
    import logging
    from kubernetes import client, config
    from kserve import (
        KServeClient,
        constants,
        V1beta1InferenceService,
        V1beta1InferenceServiceSpec,
        V1beta1PredictorSpec,
        V1beta1SKLearnSpec
    )

    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    try:
        config.load_incluster_config()
        kserve_client = KServeClient()
        
        predictor_spec = V1beta1PredictorSpec(
            min_replicas=min_replicas,
            max_replicas=max_replicas,
            service_account_name = "sa-minio-kserve",
            #sklearn=(storage_uri=model_uri)
            sklearn=V1beta1SKLearnSpec(storage_uri=model_uri, image="kserve/sklearnserver:latest" )
        )
        '''
        resource_requirements = client.V1ResourceRequirements(
        requests={"cpu": "300m"},  # Richiesta minima di 500 milliCPU
        limits={"cpu": "1"}        # Limite massimo di 1 CPU
        )'''
        inference_service = V1beta1InferenceService(
            api_version=constants.KSERVE_GROUP + '/v1beta1',
            kind=constants.KSERVE_KIND,
            metadata=client.V1ObjectMeta(name=model_name, 
                                         namespace=namespace,
                                         annotations={'sidecar.istio.io/inject':'false'}),
            spec=V1beta1InferenceServiceSpec(predictor=predictor_spec)  # Impostazione delle risorse
        )
        
        kserve_client.create(inference_service)
        logger.info(f"Model {model_name} deployed successfully in namespace {namespace}")
    
    except Exception as e:
        logger.error(f"Failed to deploy model {model_name}: {str(e)}")
        raise

# Define the pipeline
@dsl.pipeline(
    name='Ridge_MSE',
    description='Inference Service Ridge'
)
def Ridge_MSE():
    X = get_X().output
    Y = get_Y().output
    create_inference_service(
     model_uri="s3://mlpipeline/models/leafRecognize/1/",
     model_name=train_model(X=X, Y=Y).output,
     min_replicas=1,
     max_replicas=3
    )
    
#pipeline_file_path = 'Ridge_MSE.yaml' # extract it from your database
from kfp.compiler import Compiler

# Compile the pipeline
pipeline_filename = 'Ridge_MSE.yaml'
Compiler().compile(pipeline_func=Ridge_MSE, package_path=pipeline_filename)
